In [ ]:
#@Author: Gemma Gordon
#@Date: 2022
# Structural clustering using Dunbrack canonical forms (pyIgClassify2 https://www.biorxiv.org/content/10.1101/2022.10.12.511988v1)

Structural clustering output gives, for all of H1 H2 and H3 for both Abs and Nbs, the PDB structure and their cluster (a representative PDB structure). This representative structure will be matched with a canonical form. We want to know how structurally different Abs and Nbs loops are/whether they occupy different structural space. Some clusters contain both Abs and Nbs structures. To find if this is significant, we can randomly sample the clusters and find the number/proportion of structures that fall into these 'overlap' clusters by chance. We will use this as a baseline with which to compare the actual number of structures that fall into overlap clusters. 

In [1]:
import pandas as pd
import numpy as np

Load in structural clustering results as dataframe

In [2]:
results = pd.read_csv("dunbrack_clustering_results.csv").drop('Unnamed: 0', axis=1)

In [3]:
results

,pdb,cdr1_cluster,cdr2_cluster,cdr3_cluster,cdr4_cluster,Ig_ID
0,1AHW,H1-13-1,H2-10-*,H3-10-*,H4-8-1,Abs
1,1AR1,H1-13-1,H2-10-2,H3-11-*,H4-8-1,Abs
2,1BGX,H1-14-*,H2-9-*,H3-12-*,H4-8-1,Abs
3,1BJ1,H1-13-1,H2-10-1,H3-16-*,H4-8-1,Abs
4,1CZ8,H1-13-1,H2-10-1,H3-16-*,H4-8-1,Abs
...,...,...,...,...,...,...
1132,3RVW,H1-14-1,H2-9-1,NaN,NaN,Abs
1133,6CSY,H1-13-1,H2-9-1,H3-10-2,NaN,Nbs
1134,5USI,H1-13-1,H2-10-1,H3-10-1,NaN,Abs
1135,6ERX,H1-13-1,H2-10-1,NaN,NaN,Abs


Separate into H1, H2 and H3 results

In [23]:
results_h1 = results[['pdb', 'cdr1_cluster', 'Ig_ID']]
results_h2 = results[['pdb', 'cdr2_cluster', 'Ig_ID']]
results_h3 = results[['pdb', 'cdr3_cluster', 'Ig_ID']]

In [24]:
results_h1.columns

Index(['pdb', 'cdr1_cluster', 'Ig_ID'], dtype='object')

In [25]:
results_h1.to_csv('results_h1')
results_h2.to_csv('results_h2')
results_h3.to_csv('results_h3')

For the results for each loop, need to:

- identify which clusters have Nbs and Abs members in them
- find how many overlap clusters
- find how many & what proportion of structures belong to overlap clusters

In [26]:
def get_freqs(results_df, col_name):

    # get breakdown of abs vs nbs within clusters
    freq_df = results_df.groupby([col_name, 'Ig_ID']).size().reset_index()
    freq_df = freq_df.pivot(index=col_name, columns='Ig_ID')[0].fillna(0).sort_values(by='Abs', ascending=False)

    return freq_df

In [27]:
def find_overlap_clusters(results_df, col_name):

    # input results for a loop and get frequencies of Abs and Nbs
    freq_df = get_freqs(results_df, col_name)
    # identify overlap clusters 
    overlaps_id = []
    overlaps_abs, overlaps_nbs = 0, 0 

    for row in freq_df.itertuples():
        # if there are members from Abs and Nbs in the cluster
        # row[1] is Abs and row[2] is Nbs
        if row[1] != 0 and row[2] != 0:
            # row[0] is PDB ID
            overlaps_id.append(row[0])
            # how many structures in overlapping cluster?
            overlaps_abs += int(row[1])
            overlaps_nbs += int(row[2])
    
    # get proportion of total structures that are in overlap clusters
    # NOTE want proportion (1) just abs (2) just nbs (3) abs and nbs 
    overlaps_props = dict()
    overlaps_props['Abs'] = overlaps_abs / len(results_df)
    overlaps_props['Nbs'] = overlaps_nbs / len(results_df)
    overlaps_props['Both'] = (overlaps_abs + overlaps_nbs) / len(results_df)

    return overlaps_id, overlaps_props

In [28]:
overlaps_id_h1, overlaps_prop_h1 = find_overlap_clusters(results_h1, 'cdr1_cluster')
overlaps_id_h2, overlaps_prop_h2 = find_overlap_clusters(results_h2, 'cdr2_cluster')
overlaps_id_h3, overlaps_prop_h3 = find_overlap_clusters(results_h3, 'cdr3_cluster')

In [29]:
print('H1 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h1)
print('H2 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h2)
print('H3 PROPORTION OF STRUCTURES IN OVERLAP:', overlaps_prop_h3)

H1 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.6948109058927001, 'Nbs': 0.23131046613896217, 'Both': 0.9261213720316622}
H2 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.7326297273526825, 'Nbs': 0.2620932277924362, 'Both': 0.9947229551451188}
H3 PROPORTION OF STRUCTURES IN OVERLAP: {'Abs': 0.6701846965699209, 'Nbs': 0.2620932277924362, 'Both': 0.9322779243623571}


In [30]:
all_h1_freqs, all_h2_freqs, all_h3_freqs = get_freqs(results_h1, 'cdr1_cluster'), get_freqs(results_h2, 'cdr2_cluster'), get_freqs(results_h3, 'cdr3_cluster')

In [31]:
for i, loop in enumerate([all_h1_freqs, all_h2_freqs, all_h3_freqs]):

    print(i)

    #how many Abs and Nbs clusters overlap, how many clusters contain only Abs or Nbs?
    print('Total:', len(loop))
    print('Only Abs:', len(loop.loc[(loop["Abs"] != 0) & (loop["Nbs"] == 0)]))
    print('Only Nbs:', len(loop.loc[(loop["Abs"] == 0) & (loop["Nbs"] != 0)]))
    print('Overlap:', len(loop.loc[(loop["Abs"] != 0) & (loop["Nbs"] != 0)]))
    #how many clusters are singletons?
    print('Abs singletons:', len(loop.loc[(loop["Abs"] == 1) & (loop["Nbs"] == 0)]))
    print('Nbs singletons:', len(loop.loc[(loop["Nbs"] == 1) & (loop["Abs"] == 0)]))

0
Total: 21
Only Abs: 6
Only Nbs: 6
Overlap: 9
Abs singletons: 2
Nbs singletons: 4
1
Total: 15
Only Abs: 2
Only Nbs: 1
Overlap: 12
Abs singletons: 1
Nbs singletons: 1
2
Total: 35
Only Abs: 7
Only Nbs: 0
Overlap: 28
Abs singletons: 0
Nbs singletons: 0


Generate sets of random clusters that match sizes of original clusters and see how many by chance overlap in Abs and Nbs members

In [32]:
import pandas as pd
from tqdm import tqdm

def get_cluster_sizes(cluster_file, col_name):
    '''Returns array with sizes of multiple occupancy clusters'''
    df = pd.read_csv(cluster_file, index_col=0)
    cluster_size = df.groupby(col_name).nunique()['pdb'].sort_values(ascending=False)
    return cluster_size[cluster_size > 1].values

def sample_clusters_from_db(db, cluster_sizes):
    new_db = pd.DataFrame()

    for i, cluster_size in enumerate(cluster_sizes):
        cluster = db.sample(n=cluster_size, replace=False)
        cluster['cluster'] = str(i)
        new_db = pd.concat([new_db, cluster])
        db = db.drop(cluster.index)
    return new_db

def generate_random_clusters(cluster_size, n_replicates, db):
    '''Returns list of dataframes with Abs randomly assigned to clusters'''
    random_clusters = []

    db['cluster'] = 'None'

    for i in tqdm(range(n_replicates)):
        random_clusters.append(sample_clusters_from_db(db, cluster_size))

    return random_clusters

creates list of cluster sizes 

In [33]:
cluster_sizes_h1 = get_cluster_sizes('pyigclassify2/results_h1', 'cdr1_cluster')
cluster_sizes_h2 = get_cluster_sizes('pyigclassify2/results_h2', 'cdr2_cluster')
cluster_sizes_h3 = get_cluster_sizes('pyigclassify2/results_h3', 'cdr3_cluster')

In [34]:
cluster_sizes_h1

array([687, 280,  26,  25,  20,  19,  16,  13,  13,  11,   6,   6,   3,
         3,   3])

generates 20 sets of random clusters of same sizes as original cluster sizes, returns list of dfs

In [35]:
random_clusters_h1 = generate_random_clusters(cluster_sizes_h1, 20, results_h1)
random_clusters_h2 = generate_random_clusters(cluster_sizes_h2, 20, results_h2)
random_clusters_h3 = generate_random_clusters(cluster_sizes_h3, 20, results_h3)

<ipython-input-32-18ace9e4be0d>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db['cluster'] = 'None'
100%|██████████| 20/20 [00:00<00:00, 30.76it/s]


In [36]:
len(random_clusters_h1)

20

For each DF in results, find how many overlap and which overlap to find proportions

for each df, get list of overlap clusters

In [40]:
def get_overlap_ids_list(random_clusters, col_name):

    overlap_ids = []
    for rc in random_clusters:
        overlap_id_df, _ = find_overlap_clusters(rc, col_name)
        overlap_ids.append(overlap_id_df)

    return overlap_ids

In [41]:
random_overlap_ids_h1 = get_overlap_ids_list(random_clusters_h1, 'cdr1_cluster')
random_overlap_ids_h2 = get_overlap_ids_list(random_clusters_h2, 'cdr2_cluster')
random_overlap_ids_h3 = get_overlap_ids_list(random_clusters_h3, 'cdr3_cluster')

In [42]:
len(random_overlap_ids_h1)

20

Label PDB structures in results as whether overlap or non-overlap cluster

In [43]:
def add_overlap_label(results_df, overlap_ids):
    
    output_df = results_df.copy(deep=True)
    overlap_labels = []
    for row in output_df.itertuples():
        # if cluster_by_RMSD column (representative structure for cluster) is in the overlap list:
        if row[2] in overlap_ids:
            overlap_labels.append('Overlap')
        else:
            overlap_labels.append('Non-overlap')
            
    # create new column in results df
    output_df['Overlap_label'] = overlap_labels

    return output_df

In [44]:
def find_overlaps(random_clusters, overlap_ids):

    overlap_dfs = []
    for rc, ids in zip(random_clusters, overlap_ids):
        overlap_df = add_overlap_label(rc, ids)
        overlap_dfs.append(overlap_df)

    return overlap_dfs

In [45]:
random_clusters_h1_l = find_overlaps(random_clusters_h1, random_overlap_ids_h1)
random_clusters_h2_l = find_overlaps(random_clusters_h2, random_overlap_ids_h2)
random_clusters_h3_l = find_overlaps(random_clusters_h3, random_overlap_ids_h3)

In [46]:
len(random_clusters_h1_l)

20

Count how many clusters are overlap clusters in the random clusters & find the mean and stdev

for each labelled df, get freq df

In [47]:
random_clusters_h1_l[0].loc[(random_clusters_h1_l[0]['Overlap_label'] == 'Overlap')]

,pdb,cdr1_cluster,Ig_ID,cluster,Overlap_label
65,2DQG,H1-13-1,Abs,0,Overlap
937,5J57,H1-13-*,Nbs,0,Overlap
343,5E94,H1-14-*,Abs,0,Overlap
130,3LZF,H1-15-*,Abs,0,Overlap
241,4L5F,H1-13-1,Abs,0,Overlap
...,...,...,...,...,...
749,7KMI,H1-13-1,Abs,13,Overlap
681,7BEH,H1-13-1,Abs,13,Overlap
1123,7OAQ,H1-13-*,Nbs,14,Overlap
867,4FHB,H1-13-1,Nbs,14,Overlap


find total number of clusters which are randomly overlap clusters

In [52]:
def get_overlap_summary(dfs, col_name):

    overlap_counts = []
    for df in dfs:
        #how many Abs and Nbs clusters overlap?
        overlap_df = df[[col_name, 'Overlap_label']].drop_duplicates(subset=col_name, keep='first')
        overlap_count = len(overlap_df.loc[(overlap_df['Overlap_label'] == 'Overlap')])
        overlap_counts.append(overlap_count)

    # get mean and stdev overlap
    mean_overlap = np.mean(overlap_counts)
    stdev_overlap = np.std(overlap_counts)

    summary = {'MEAN': mean_overlap, 'STDEV': stdev_overlap}

    return summary

In [53]:
h1_summary = get_overlap_summary(random_clusters_h1_l, 'cdr1_cluster')
h2_summary = get_overlap_summary(random_clusters_h2_l, 'cdr2_cluster')
h3_summary = get_overlap_summary(random_clusters_h3_l, 'cdr3_cluster')

In [54]:
print('H1 RANDOM OVERLAP RESULTS:', h1_summary)
print('H2 RANDOM OVERLAP RESULTS:', h2_summary)
print('H3 RANDOM OVERLAP RESULTS:', h3_summary)

H1 RANDOM OVERLAP RESULTS: {'MEAN': 9.0, 'STDEV': 0.0}
H2 RANDOM OVERLAP RESULTS: {'MEAN': 12.0, 'STDEV': 0.0}
H3 RANDOM OVERLAP RESULTS: {'MEAN': 27.95, 'STDEV': 0.21794494717703367}
